In [21]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow_model_optimization as tfmot
import tensorflow_addons as tfa

In [32]:
inputPath = r"D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))



for i in filesArray:
    print(i)
    
max_length = 1000
df = []
labels = []
for filename in filesArray:
    data = np.loadtxt(filename, dtype=np.float32)
    
    data = data/2 #Normalise

    if len(data) < max_length:
        # pad the time series with zeros to a length of 1000
        data = np.pad(data, [(0, max_length - len(data)), (0, 0)], mode='constant')
    elif len(data) > max_length:
        # truncate the time series to a length of 1000
        data = data[:max_length, :]
    a = filename.split("\\")[-2]
    if a == "True":
        label = [1]
    else:
        label = [0]
    labels.append(label)
    df.append(data)

df_array = np.array(df)
labels_array = np.array(labels)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

print(df_array.shape)
print(labels_array.shape)

['False' 'True']
2
1810
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_191958.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_192022.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_192026.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_192043.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_192126.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_192132.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_192140.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data\False\data_20230510_192153.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data

In [42]:
import datetime

# Define the log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        print(0.001 * tf.math.exp(-0.02*((epoch-10))))
        return 0.001 * tf.math.exp(-0.02*((epoch-10)))

# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    
total_epochs = 0

model = Sequential()

model.add(L.Input(shape=(1000, 1)))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=1, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=2, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=4, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=8, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=16, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=32, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=64, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=100, activation='relu'))
model.add(L.GlobalAveragePooling1D())
model.add(L.Flatten())
model.add(L.Dropout(0.2))  # Dropout layer to reduce overfitting
model.add(L.Dense(256, activation='relu'))  # Additional dense layer
model.add(L.Dropout(0.2))  # Dropout layer to reduce overfitting
model.add(L.Dense(1, activation='sigmoid'))


model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])

# model.build((235, 4001, 1))
print(model.summary())

new_epochs = 190
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.15, batch_size = 32, callbacks=[tensorboard_callback,lr_scheduler])
total_epochs += new_epochs

model.evaluate(df_array, labels_array, batch_size=1)

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_164 (Conv1D)         (None, 996, 64)           384       
                                                                 
 conv1d_165 (Conv1D)         (None, 988, 64)           20544     
                                                                 
 conv1d_166 (Conv1D)         (None, 972, 64)           20544     
                                                                 
 conv1d_167 (Conv1D)         (None, 940, 64)           20544     
                                                                 
 conv1d_168 (Conv1D)         (None, 876, 64)           20544     
                                                                 
 conv1d_169 (Conv1D)         (None, 748, 64)           20544     
                                                                 
 conv1d_170 (Conv1D)         (None, 492, 64)         

In [151]:
# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer=tf.optimizers.Adam(learning_rate=   0.00005),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])

new_epochs = 10
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.05, batch_size = 1, callbacks=[tensorboard_callback])
total_epochs += new_epochs

Epoch 187/196
1818/1818 [==============================] - 37s 20ms/step - loss: 0.0089 - binary_accuracy: 0.9868 - val_loss: 0.0202 - val_binary_accuracy: 0.9688
Epoch 188/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0063 - binary_accuracy: 0.9917 - val_loss: 0.0301 - val_binary_accuracy: 0.9792
Epoch 189/196
1818/1818 [==============================] - 38s 21ms/step - loss: 0.0047 - binary_accuracy: 0.9945 - val_loss: 0.0260 - val_binary_accuracy: 0.9896
Epoch 190/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0066 - binary_accuracy: 0.9917 - val_loss: 0.0381 - val_binary_accuracy: 0.9792
Epoch 191/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0049 - binary_accuracy: 0.9934 - val_loss: 0.0236 - val_binary_accuracy: 0.9792
Epoch 192/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0056 - binary_accuracy: 0.9934 - val_loss: 0.0219 - val_binary_accuracy: 0.9792
Epoch 193/196
1818/181

In [110]:
total_epochs = 126

In [29]:
model.evaluate(df_array, labels_array, batch_size=1)

1810/1810 [==============================] - 6s 3ms/step - loss: 0.0100 - binary_accuracy: 0.9923


[0.009952179156243801, 0.9922651648521423]

In [31]:
model.evaluate(df_array, labels_array, batch_size=1)

1914/1914 [==============================] - 6s 3ms/step - loss: 0.0100 - binary_accuracy: 0.9922


[0.009992574341595173, 0.9921630024909973]

In [28]:
tf.keras.models.save_model(model, 'model', save_format='tf')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [44]:

# Load your Keras model
model = tf.keras.models.load_model('model')

# Define the pruning parameters
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.1,
    final_sparsity=0.99,
    begin_step=0,
    end_step=10
)

# Define the pruning callback
pruning_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='/tmp')
]

# Create a pruned model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)

# Train the pruned model
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])
pruned_model.fit(df_array, labels_array, shuffle=1, epochs=10, validation_split=0.25, callbacks=pruning_callbacks)

# Remove the pruning wrappers from the model
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Save the pruned model
pruned_model.save('pruned_model',  save_format='tf')

Epoch 1/10
41/41 [==============================] - 8s 147ms/step - loss: 0.0120 - binary_accuracy: 0.9847 - val_loss: 0.0038 - val_binary_accuracy: 0.9931
Epoch 2/10
41/41 [==============================] - 6s 143ms/step - loss: 0.0105 - binary_accuracy: 0.9870 - val_loss: 0.0038 - val_binary_accuracy: 0.9931
Epoch 3/10
41/41 [==============================] - 6s 150ms/step - loss: 0.0114 - binary_accuracy: 0.9831 - val_loss: 0.0044 - val_binary_accuracy: 0.9931
Epoch 4/10
41/41 [==============================] - 6s 152ms/step - loss: 0.0056 - binary_accuracy: 0.9931 - val_loss: 0.0035 - val_binary_accuracy: 0.9931
Epoch 5/10
41/41 [==============================] - 6s 144ms/step - loss: 0.0074 - binary_accuracy: 0.9923 - val_loss: 0.0037 - val_binary_accuracy: 0.9931
Epoch 6/10
41/41 [==============================] - 6s 145ms/step - loss: 0.0109 - binary_accuracy: 0.9816 - val_loss: 0.0041 - val_binary_accuracy: 0.9954
Epoch 7/10
41/41 [==============================] - 6s 144ms/ste

INFO:tensorflow:Assets written to: pruned_model\assets


INFO:tensorflow:Assets written to: pruned_model\assets


In [46]:
# Load your Keras model
pruned_model = tf.keras.models.load_model('pruned_model')
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])
pruned_model.evaluate(df_array, labels_array, batch_size=1)

1738/1738 [==============================] - 4s 2ms/step - loss: 0.0055 - binary_accuracy: 0.9937


[0.0055006081238389015, 0.9936708807945251]

In [47]:
converter = tf.lite.TFLiteConverter.from_saved_model("pruned_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


with open('model_quantized.tflite', 'wb') as f:
  f.write(tflite_model)

In [48]:
interpreter = tf.lite.Interpreter(model_path="model_quantized.tflite")
interpreter.allocate_tensors()

In [49]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

In [51]:
print(input_details[0]['shape'])
print(df_array.shape)
#old_df = df_array

[   1 1000    1]
(1738, 1000)


In [52]:

df_array = np.expand_dims(old_df, axis=-1) # add extra dimension to the end

In [56]:
correct = 0
total = 0
from scipy.stats import pearsonr


for i in range(len(df_array)):
    interpreter.set_tensor(input_details[0]['index'], df_array[i:i+1])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    origional = model.predict(df_array[i:i+1], verbose=0)
        
    print(f'Label: {labels_array[i]}    tf model prediction: {origional}    tflite model prediction: {output}')
    prediction = np.argmax(output)
    mape = abs(origional - output)
    print(mape)
    if(mape < 0.005):
            correct += 1
    
    total += 1
    print(f'{correct} of {total} predictions were accurate to within 0.5%')

accuracy = correct / total
print('Accuracy:', accuracy)

Label: [1]    tf model prediction: [[0.95268923]]    tflite model prediction: [[0.9515475]]
[[0.00114173]]
1 of 1 predictions were accurate to within 0.5%
Label: [1]    tf model prediction: [[0.7927143]]    tflite model prediction: [[0.797811]]
[[0.00509667]]
1 of 2 predictions were accurate to within 0.5%
Label: [0]    tf model prediction: [[0.03293535]]    tflite model prediction: [[0.03296516]]
[[2.9809773e-05]]
2 of 3 predictions were accurate to within 0.5%
Label: [0]    tf model prediction: [[0.03719188]]    tflite model prediction: [[0.0373679]]
[[0.00017602]]
3 of 4 predictions were accurate to within 0.5%
Label: [0]    tf model prediction: [[0.03314153]]    tflite model prediction: [[0.03311322]]
[[2.8312206e-05]]
4 of 5 predictions were accurate to within 0.5%
Label: [1]    tf model prediction: [[0.9641621]]    tflite model prediction: [[0.9622366]]
[[0.00192553]]
5 of 6 predictions were accurate to within 0.5%
Label: [0]    tf model prediction: [[0.05187451]]    tflite model

In [59]:
# Get the tensor details
tensor_details = interpreter.get_tensor_details()

# Calculate the total RAM required
total_ram = 0
for tensor in tensor_details:
    if "bytes" in tensor:
            total_ram += tensor["bytes"]

print(f"Total RAM required: {total_ram} bytes")


Total RAM required: 0 bytes
